In [136]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_breast_cancer

In [5]:
np.random.seed(123)
torch.manual_seed(123)

In [7]:
ds = load_breast_cancer()

In [20]:
dsPreditores = np.array(ds.data,dtype='float32')

In [26]:
dsPreditores[:2], dsPreditores.shape

(array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
         3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
         8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
         3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
         1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
         8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
         3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
         1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
         1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02]],
       dtype=float32),
 (569, 30))

In [72]:
dsAlvo = np.array(ds.target, dtype='float32')

In [75]:
dsAlvo[:2], dsAlvo.shape

(array([0., 0.], dtype=float32), (569,))

# Ao utilizarmos o skorch em outro notebook 'deepLearnValidacaoCruzadaTunning.ipynb'
# o GriddSearh retornou o valor 8 para hiperparametro (numero de neuronios)
# e ReLU como m função de ativação

In [98]:
# Arquitetura da rde neural 30 -> 8 -> 8 -> 1

class classificadorTorch(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense0 = nn.Linear(30, 8)
    torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std = 0.05)
      
    self.dense1 = nn.Linear(8, 8)
    torch.nn.init.normal_(self.dense1.weight, mean = 0.0, std = 0.05)
      
    self.dense2 = nn.Linear(8, 1)
      
    self.activation = nn.ReLU() # como a função de ativação será a mesma para os neuronios das 2 camadas ocultas
      
    self.dropout = nn.Dropout(0.2) # o dropout será o mesmo para ambas as camadas
      
    self.output = nn.Sigmoid() #função de ativação da saida  

  def forward(self, entradas):
    #entrada e primeira camada
    elemen = self.dense0(entradas)
    elemen = self.activation(elemen)
    elemen = self.dropout(elemen)
      
    #primeira camada e 2 camada
    elemen = self.dense1(elemen)
    elemen = self.activation(elemen)
    elemen = self.dropout(elemen)
      
    # 2 camda e camada de saida
    elemen = self.dense2(elemen)
    elemen = self.output(elemen)
    return elemen

In [99]:
classificador = classificadorTorch()

In [100]:
criterio = nn.BCELoss()
otimizador = torch.optim.Adam(classificador.parameters(), lr = 0.001,
                             weight_decay = 0.0001)

In [101]:
dsPreditores_tensor = torch.tensor(dsPreditores, dtype = torch.float)
dsAlvo_tensor       = torch.tensor(dsAlvo, dtype = torch.float)


/tmp/ipykernel_6989/1405681254.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dsAlvo_tensor       = torch.tensor(dsAlvo, dtype = torch.float)


In [102]:
dsPreditores_tensor.shape, dsAlvo_tensor.shape

(torch.Size([569, 30]), torch.Size([569, 1]))

In [103]:
dsAlvo_tensor

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
      

In [88]:
dsAlvo_tensor_transformada = dsAlvo_tensor.unsqueeze(1)
dsAlvo_tensor_transformada

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
      

In [104]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(dsPreditores_tensor, dsAlvo_tensor_transformada), batch_size = 10, shuffle = True)

#Treinamento do modelo

In [105]:
for epoca in range(100):
    totalizadorErro = 0.

    for data in train_loader:
        inputs, labels = data
        otimizador.zero_grad()

        previsao = classificador(inputs)
        loss = criterio(previsao, labels)
        loss.backward()
        otimizador.step()

        totalizadorErro += loss.item()

    print('Época %3d: perda %.5f' % (epoca+1, totalizadorErro/len(train_loader)))

Época   1: perda 1.29312
Época   2: perda 0.65192
Época   3: perda 0.59024
Época   4: perda 0.53303
Época   5: perda 0.45126
Época   6: perda 0.38766
Época   7: perda 0.36055
Época   8: perda 0.35363
Época   9: perda 0.35558
Época  10: perda 0.34723
Época  11: perda 0.32491
Época  12: perda 0.31190
Época  13: perda 0.31246
Época  14: perda 0.30331
Época  15: perda 0.27837
Época  16: perda 0.26697
Época  17: perda 0.25930
Época  18: perda 0.27977
Época  19: perda 0.25904
Época  20: perda 0.26632
Época  21: perda 0.25333
Época  22: perda 0.26931
Época  23: perda 0.27106
Época  24: perda 0.26259
Época  25: perda 0.25237
Época  26: perda 0.24799
Época  27: perda 0.24302
Época  28: perda 0.25252
Época  29: perda 0.24511
Época  30: perda 0.24010
Época  31: perda 0.24196
Época  32: perda 0.24316
Época  33: perda 0.23802
Época  34: perda 0.23537
Época  35: perda 0.25128
Época  36: perda 0.21235
Época  37: perda 0.23916
Época  38: perda 0.22229
Época  39: perda 0.23716
Época  40: perda 0.21197


In [106]:
registro = novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [107]:
classificador.eval() #parar treinamento da rede neural

classificadorTorch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [124]:
classificacao =classificador(registro)

In [125]:
classificacao 

tensor([[0.]], grad_fn=<SigmoidBackward0>)

In [138]:
dsPreditor_tensor_2 = torch.tensor(dsPreditores,dtype=torch.float)
classificacaoes = classificador.forward(dsPreditor_tensor_2).detach()

In [139]:
classificacaoes

tensor([[1.0647e-08],
        [4.2788e-05],
        [3.8576e-04],
        [8.7060e-01],
        [7.3459e-02],
        [4.0478e-01],
        [5.2558e-04],
        [8.7621e-02],
        [6.0298e-01],
        [1.7075e-01],
        [4.3002e-02],
        [1.6116e-03],
        [1.3502e-01],
        [9.6491e-01],
        [9.6491e-01],
        [1.4997e-01],
        [5.7760e-03],
        [1.4639e-03],
        [3.5258e-10],
        [9.6491e-01],
        [9.6491e-01],
        [9.9868e-01],
        [4.0498e-01],
        [6.5602e-11],
        [2.5092e-11],
        [3.8938e-04],
        [4.9391e-01],
        [2.6135e-02],
        [1.2530e-03],
        [2.7750e-01],
        [6.1220e-05],
        [8.4710e-03],
        [2.9019e-03],
        [3.0200e-05],
        [8.9624e-03],
        [4.0077e-02],
        [9.6201e-01],
        [9.9777e-01],
        [8.3696e-01],
        [9.6491e-01],
        [6.1109e-01],
        [9.5098e-01],
        [2.2278e-04],
        [6.6542e-02],
        [8.6468e-01],
        [1

In [140]:
classificacaoesBooleanas = (classificacaoes >= .5)

In [141]:
classificacaoesBooleanas

tensor([[False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [ True

In [142]:
accuracy_score(load_breast_cancer().target,classificacaoesBooleanas.numpy())

0.9244288224956063

In [143]:

#calculo do erro total para o dataset
F.binary_cross_entropy(classificacaoes,dsAlvo_tensor)

tensor(0.2026)

Salvar classificador 

In [145]:
classificador.state_dict()

OrderedDict([('dense0.weight',
              tensor([[ 2.4747e-01,  1.6270e-01,  1.4376e-01,  5.4507e-02,  1.8270e-01,
                       -4.7519e-01, -9.3101e-01, -6.4578e-01,  2.8544e-01,  2.2954e-01,
                        2.2214e-01,  3.0199e-01, -1.6901e-01, -6.1613e-02,  2.1505e-02,
                       -2.0054e-01, -2.4252e-01, -6.4089e-02,  1.3147e-01,  1.9480e-02,
                        1.8146e-01,  5.1657e-02,  1.8332e-01, -1.0858e-01,  1.8800e-01,
                       -7.3896e-01, -9.6684e-01, -7.2951e-01,  1.3708e-01,  7.7851e-02],
                      [-6.4776e-03, -1.3625e-02, -1.8083e-02, -1.5228e-02,  6.9908e-39,
                       -7.3840e-40, -7.9915e-40, -3.4986e-39, -3.3780e-39, -2.9929e-39,
                       -1.5505e-30, -1.6897e-07, -2.2585e-05, -1.2392e-02, -1.7058e-40,
                        3.8059e-40,  5.7697e-39, -5.4802e-39, -5.7323e-39,  4.5979e-39,
                       -6.5702e-03, -1.7202e-02, -1.9240e-02, -1.6675e-02, -5.2512e-39,


#salvando rede neural em arquivo

In [144]:
torch.save(classificador.state_dict(), 'checkpoint.pth') #salvando rede neural